<a href="https://colab.research.google.com/github/darkengross/google_gemini_based_invoice_data_extraction/blob/main/google_gemini_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-google-genai

In [ ]:
!pip install -qU langchain-google-vertexai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 6.0 MB/s eta 0:00:00


In [ ]:
!pip install pypdf
!pip install langchain-community
!pip install tiktoken
!pip install pdfminer.six
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.7 MB/s eta 0:00:00


In [ ]:
import google.generativeai as genai
from google.colab import userdata
import textwrap

In [ ]:
import pypdf as pdf
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PDFMinerLoader, TextLoader, Docx2txtLoader
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain_community.llms import Cohere
# from langchain_community.vectorstores import FAISS
from langchain.vectorstores import Chroma
import datetime, json, os, tiktoken
from IPython.display import Markdown
from PIL import Image

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
import os
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key = GOOGLE_API_KEY)
GOOGLE_APPLICATION_CREDENTIALS = userdata.get('GOOGLE_APPLICATION_CREDENTIALS')

def to_markdown(text):
  # text = text.replace(".", " *")
  return Markdown(textwrap.indent(text, '> ', predicate = lambda _:True))


In [ ]:
model = genai.GenerativeModel(model_name = "gemini-1.5-flash")
model

genai.GenerativeModel(
    model_name='models/gemini-1.5-flash',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)

## Images

In [ ]:
myfile = genai.upload_file("pos_invoice.jpg")
print(f"{myfile=}")


myfile=genai.File({
    'name': 'files/xl8ibpvpsb32',
    'display_name': 'pos_invoice.jpg',
    'mime_type': 'image/jpeg',
    'sha256_hash': 'OGU5YTk2ZTMwMGEwNWY2ZWY3NzYzZjAxMTZiNzU0MTgzNzM5YTg3MGU0NjYzOGIwNGU1ODNmM2ZiNTEwN2FlNw==',
    'size_bytes': '538694',
    'state': 'ACTIVE',
    'uri': 'https://generativelanguage.googleapis.com/v1beta/files/xl8ibpvpsb32',
    'create_time': '2024-11-21T07:26:16.379301Z',
    'expiration_time': '2024-11-23T07:26:16.366314831Z',
    'update_time': '2024-11-21T07:26:16.379301Z'})


In [ ]:
result = model.generate_content(
    [myfile, "\n\n",
     """Return all the product details in a array of json objects.Find the fields that are available in the image. Each object should have the folowing format:
     {
         Description: Name of the product,
         Rate: Rate of that product,
         Quantity: Quantity of that product,
         Taxable Value: Taxable value of that product,
         GST: GST of that product,
         Amount: Amount of that product,
     }
     """]
)
print(f"{result.text=}")

result.text='```json\n[\n  {\n    "Description": "iPHONE 16",\n    "Rate": 79990.00,\n    "Quantity": 1.0,\n    "Taxable Value": 74390.7,\n    "GST": null,\n    "Amount": 69183.35\n  },\n  {\n    "Description": "iPHONE 16 Cover",\n    "Rate": 4599.00,\n    "Quantity": 1.0,\n    "Taxable Value": 4277.07,\n    "GST": null,\n    "Amount": 3977.68\n  },\n  {\n    "Description": "Beats PRO X",\n    "Rate": 24999.00,\n    "Quantity": 1.0,\n    "Taxable Value": 23249.07,\n    "GST": null,\n    "Amount": 21621.64\n  }\n]\n```'


In [ ]:
products = result.text
to_markdown(products)

> ```json
> [
>   {
>     "Description": "iPHONE 16",
>     "Rate": 79990.00,
>     "Quantity": 1.0,
>     "Taxable Value": 74390.7,
>     "GST": null,
>     "Amount": 69183.35
>   },
>   {
>     "Description": "iPHONE 16 Cover",
>     "Rate": 4599.00,
>     "Quantity": 1.0,
>     "Taxable Value": 4277.07,
>     "GST": null,
>     "Amount": 3977.68
>   },
>   {
>     "Description": "Beats PRO X",
>     "Rate": 24999.00,
>     "Quantity": 1.0,
>     "Taxable Value": 23249.07,
>     "GST": null,
>     "Amount": 21621.64
>   }
> ]
> ```

In [ ]:
invoices = model.generate_content(
    [myfile, "\n\n",
     """Return all the product details in a array of json objects.Find the fields that are available. Each object should have the folowing format:
     {
         Name: Name of the buyer (also called Beneficiary name),
         Serial Number: Serial number of the invoice(also called bill number ),
         products: [
          {
            name: name of product,
            quantity: quantity of product,
            taxable amount: Taxable amount of product,

          },
          {
            name: name of product,
            quantity: quantity of product,
            taxable amount: Taxable amount of product,
          },
         ] A list of all the products
         Date: Date of the bill,
         Total Amount: Amount of the full bill,
     }
     """]
)
print(f"{invoices.text=}")

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2923.33ms


invoices.text='```json\n[\n  {\n    "Name": "Shounak",\n    "Serial Number": "RAY/23-24/286",\n    "products": [\n      {\n        "name": "iPHONE 16",\n        "quantity": 1.0,\n        "taxable amount": 79990.00\n      },\n      {\n        "name": "iPHONE 16 Cover",\n        "quantity": 1.0,\n        "taxable amount": 4599.00\n      },\n      {\n        "name": "Beats PRO X",\n        "quantity": 1.0,\n        "taxable amount": 24999.00\n      }\n    ],\n    "Date": "12 Nov 2024",\n    "Total Amount": 94782.67\n  }\n]\n```\n'


In [ ]:
to_markdown(invoices.text)

> ```json
> [
>   {
>     "Name": "Shounak",
>     "Serial Number": "RAY/23-24/286",
>     "products": [
>       {
>         "name": "iPHONE 16",
>         "quantity": 1.0,
>         "taxable amount": 79990.00
>       },
>       {
>         "name": "iPHONE 16 Cover",
>         "quantity": 1.0,
>         "taxable amount": 4599.00
>       },
>       {
>         "name": "Beats PRO X",
>         "quantity": 1.0,
>         "taxable amount": 24999.00
>       }
>     ],
>     "Date": "12 Nov 2024",
>     "Total Amount": 94782.67
>   }
> ]
> ```


In [ ]:
Img_customer_details = model.generate_content(
    [myfile, "\n\n",
      """Return all the product details in a array of json objects.Find the fields that are available. Each object should have the folowing format:
      {
          Customer Name: Name of the buyer (also called Beneficiary name),
          Phone number: Phone Number of customer,
          Total amount: total amount of the invoice,
          Serial Number: Serial number of the invoice(also called bill number ),
          Date: Date of the bill,
      }
      """
    ],
)

print(f"{Img_customer_details.text=}")

Img_customer_details.text='```json\n[\n  {\n    "Customer Name": "Shounak",\n    "Phone number": "3562472477",\n    "Total amount": "94,782.67",\n    "Serial Number": "RAY/23-24/286",\n    "Date": "12 Nov 2024"\n  }\n]\n```'


In [ ]:
to_markdown(Img_customer_details.text)

> ```json
> [
>   {
>     "Customer Name": "Shounak",
>     "Phone number": "3562472477",
>     "Total amount": "94,782.67",
>     "Serial Number": "RAY/23-24/286",
>     "Date": "12 Nov 2024"
>   }
> ]
> ```

## Excel

In [ ]:
import pandas as pd
excel_file_path = "Transaction-details.xlsx"
df = pd.read_excel(excel_file_path)

# Step 2: Convert Excel Data to JSON
excel_data = df.to_dict(orient="records")
excel_data_text = json.dumps(excel_data, indent=4)  # Convert to readable JSON format


In [ ]:

# Step 3: Create Prompt for Model
prompt_template = f"""
The following is data extracted from an Excel file:
{excel_data_text}

Return all the product details in an array of JSON objects. Identify the fields available in the data and organize them into the following format:
[
    {{
        "Description": "Name of the product",
        "Rate": "Rate of that product",
        "Quantity": "Quantity of that product",
        "Taxable Value": "Taxable value of that product",
        "GST": "GST of that product",
        "Amount": "Amount of that product"
    }}
]

Do not give anything other than the json output
"""

# Step 4: Use Google Generative AI to Process Data
# model = genai.ChatModel(model="gemini-pro", temperature=0.3)
try:
    response = model.generate_content(prompt_template)
    print("Generated Output:\n", response.text)
except Exception as e:
    print(f"Error: {e}")

Error: HTTPConnectionPool(host='localhost', port=38709): Read timed out. (read timeout=600.0)


In [ ]:
prompt_template = f"""
The following is data extracted from an Excel file:
{excel_data_text}

Return all the product details in an array of JSON objects. Identify the fields available in the data and organize them into the following format:
[
    Name: Name of the buyer (also called Beneficiary name),
    Serial Number: Serial number of the invoice(also called bill number ),
    products: [
    {{
      name: name of product,
      quantity: quantity of product,
      taxable amount: Taxable amount of product,

    }},
    {{
      name: name of product,
      quantity: quantity of product,
      taxable amount: Taxable amount of product,
    }},
    ] A list of all the products
    Date: Date of the bill,
    Total Amount: Amount of the full bill,
]

Do not give anything other than the json output
"""

# Step 4: Use Google Generative AI to Process Data
# model = genai.ChatModel(model="gemini-pro", temperature=0.3)
try:
    response = model.generate_content(prompt_template)
    print("Generated Output:\n", response.text)
except Exception as e:
    print(f"Error: {e}")

Generated Output:
 ```json
[
  {
    "Name": "Shounak",
    "Serial Number": "RAY/23-24/286",
    "products": [
      {
        "name": "iPHONE 16",
        "quantity": "1.000",
        "taxable amount": 66000
      },
      {
        "name": "iPHONE 16 Cover",
        "quantity": "1.000",
        "taxable amount": 3914.49
      },
      {
        "name": "Beats PRO X",
        "quantity": "1.000",
        "taxable amount": 21345.91
      }
    ],
    "Date": "12 Nov 2024",
    "Total Amount": 71260.4
  },
  {
    "Name": "Abhinav",
    "Serial Number": "RAY/23-24/285",
    "products": [
      {
        "name": "SPEAKER",
        "quantity": "1.000",
        "taxable amount": 10000
      }
    ],
    "Date": "12 Nov 2024",
    "Total Amount": 10000
  },
  {
    "Name": "Ramesh",
    "Serial Number": "RAY/23-24/284",
    "products": [
      {
        "name": "12 MM PLAIN GLASS",
        "quantity": "1.000",
        "taxable amount": 88.98
      }
    ],
    "Date": "08 Nov 2024",
    "T

In [ ]:
prompt_template = f"""
The following is data extracted from an Excel file:
{excel_data_text}

Return all the product details in an array of JSON objects. Identify the fields available in the data and organize them into the following format:
[
    {{
        "Customer": "Name of the customer",
        "Total Amount": "Rate of invoice",
        "serial Number": "Serial number of the invoice(also called bill number )",
        "Date": "Date of purchase",
        "Phone Number": "Phone number of customer"
    }}
]

Do not give anything other than the json output
"""

# Step 4: Use Google Generative AI to Process Data
# model = genai.ChatModel(model="gemini-pro", temperature=0.3)
try:
    response = model.generate_content(prompt_template)
    print("Generated Output:\n", response.text)
except Exception as e:
    print(f"Error: {e}")

## pdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 61.6 MB/s eta 0:00:00


In [ ]:
sample_pdf = genai.upload_file("simple_invoice.pdf")


In [ ]:
pdf_products = model.generate_content([sample_pdf, "\n\n",
     """Return all the product details in a array of json objects.Find the fields that are available in the image. Each object should have the folowing format:
     {
         Description: Name of the product,
         Rate: Rate of that product,
         Quantity: Quantity of that product,
         Taxable Value: Taxable value of that product,
         GST: GST of that product,
         Amount: Amount of that product,
     }
     """])
print(pdf_products.text)

```json
[
  {
    "Description": "GEMS CHOCLATE POUCH",
    "Rate": 4.7619,
    "Quantity": 1000.000,
    "Taxable Value": 4761.90,
    "GST": "238.10 (5%)",
    "Amount": 5000.00
  },
  {
    "Description": "TREAT BKS CASE 80PKT",
    "Rate": 535.7143,
    "Quantity": 50.000,
    "Taxable Value": 26785.71,
    "GST": "3214.29 (12%)",
    "Amount": 30000.00
  },
  {
    "Description": "NUTRI CHOICE BKS CASE",
    "Rate": 666.6667,
    "Quantity": 25.000,
    "Taxable Value": 16666.67,
    "GST": "833.33 (5%)",
    "Amount": 17500.00
  },
  {
    "Description": "MILK BIKIS CLASSIC CASE 120PK",
    "Rate": 809.5238,
    "Quantity": 20.000,
    "Taxable Value": 16190.48,
    "GST": "809.52 (5%)",
    "Amount": 17000.00
  }
]
```


In [ ]:
pdf_invoices = model.generate_content(
    [sample_pdf, "\n\n",
     """Return all the product details in a array of json objects.Find the fields that are available. Each object should have the folowing format:
     {
         Name: Name of the buyer (also called Beneficiary name),
         Serial Number: Serial number of the invoice(also called bill number ),
         products: [
          {
            name: name of product,
            quantity: quantity of product,
            taxable amount: Taxable amount of product,

          },
          {
            name: name of product,
            quantity: quantity of product,
            taxable amount: Taxable amount of product,
          },
         ] A list of all the products
         Date: Date of the bill,
         Total Amount: Amount of the full bill,
     }
     """]
)
print(f"{pdf_invoices.text=}")

In [ ]:
pdf_customer_details = model.generate_content(
    [sample_pdf, "\n\n",
      """Return all the product details in a array of json objects.Find the fields that are available. Each object should have the folowing format:
      {
          Customer Name: Name of the buyer (also called Beneficiary name),
          Phone number: Phone Number of customer,
          Total amount: total amount of the invoice,
          Serial Number: Serial number of the invoice(also called bill number ),
          Date: Date of the bill,
      }
      """
    ],
)

print(f"{pdf_customer_details.text=}")